In [1]:
import boto3
import pandas as pd; pd.set_option('display.max_column', 100)
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler

from scipy.stats import boxcox
from precision_recall_cutoff import precision_recall_cutoff
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE, RFECV
from sklearn.svm import SVC

from sklearn.metrics import recall_score, classification_report

## define bucket in which you are trying to reach
s3 = boto3.resource('s3')
bucket_name = 'daltondencklau-data445-bucket'
bucket = s3.Bucket(bucket_name)

## define csv file to read in the bucket
file_key= 'turnover.csv'

## syntax to allow us to read the file
bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## reading the data file
turnover = pd.read_csv(file_content_stream)
turnover.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [2]:
## changing 'sales' and 'salary' to dummy variables
turnover = pd.concat([turnover.drop(columns = ['sales', 'salary'], axis = 1), pd.get_dummies(turnover[['sales', 'salary']])], axis = 1)
turnover.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales_IT,sales_RandD,sales_accounting,sales_hr,sales_management,sales_marketing,sales_product_mng,sales_sales,sales_support,sales_technical,salary_high,salary_low,salary_medium
0,0.38,0.53,2,157,3,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0
1,0.80,0.86,5,262,6,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,0.11,0.88,7,272,4,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
3,0.72,0.87,5,223,5,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0
4,0.37,0.52,2,159,3,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0


In [3]:
## Engineering Features from the decision tree model
turnover['interaction_1'] = np.where(((turnover['satisfaction_level'] >= 0.115) &
                                      (turnover['satisfaction_level'] <= 0.465) &
                                      (turnover['number_project'] > 2.5)), 1, 0)
                              
turnover['interaction_2'] = np.where(((turnover['satisfaction_level'] <= 0.465) &
                                      (turnover['satisfaction_level'] <= 2.5) &
                                      (turnover['number_project'] <= 0.575)), 1, 0)

turnover['interaction_3'] = np.where(((turnover['satisfaction_level'] > 0.465) &
                                      (turnover['time_spend_company'] <= 0.45) &
                                      (turnover['average_montly_hours'] <= 290.5)), 1, 0)

In [4]:
## defining input and target
x = turnover[['satisfaction_level', 'number_project', 'time_spend_company', 'interaction_1','interaction_3']]
y = turnover['left']

## splitting the data into training and testing
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify = y)

In [5]:
## scaling the data
scaler = MinMaxScaler()

x_train = pd.DataFrame(scaler.fit_transform(x_train), columns = x_train.columns)
x_test = pd.DataFrame(scaler.fit_transform(x_test), columns = x_test.columns)

### GridsearchCV with Random Forest
***

In [6]:
## defining hyperparameters to consider tuning
rf_param_grid = {'n_estimators': [100, 300, 500],
                 'min_samples_split': [10, 15],
                 'min_samples_leaf': [5, 7],
                 'max_depth': [3, 5, 7]}

## running GridsearchCV
rf_grid_search = GridSearchCV(estimator = RandomForestClassifier(), param_grid = rf_param_grid, 
                              cv = 5, scoring = 'f1').fit(x_train, y_train)

## extracting the best hyperparameter combinationsRF_param_grid
rf_grid_search.best_params_

{'max_depth': 7,
 'min_samples_leaf': 5,
 'min_samples_split': 10,
 'n_estimators': 100}

In [7]:
## building random forest model
rf_md = RandomForestClassifier(n_estimators = 100, max_depth = 7,
                               min_samples_leaf = 5, min_samples_split = 10).fit(x_train, y_train)

## predicting on the testing
rf_preds = rf_md.predict_proba(x_test)[:, 1]

## changing likelihoods to labes
rf_labels = precision_recall_cutoff(y_test, rf_preds)

## printing classification report
print(classification_report(y_test, rf_labels))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98      2286
           1       0.95      0.90      0.93       714

    accuracy                           0.97      3000
   macro avg       0.96      0.95      0.95      3000
weighted avg       0.97      0.97      0.97      3000



### Gridsearch CV with SVM
***

In [10]:
## defining hyperparameters to consider tuning
svm_param_grid = {'kernel': ['rbf', 'poly', 'sigmoid'],
                 'C': [0.01, 0.1, 1, 10],
                 'gamma': [0.001, 0.01, 0.1, 1]}

## running grid search cv
svm_grid_search = GridSearchCV(estimator = SVC(probability = True), param_grid = svm_param_grid, cv = 5, scoring = 'f1').fit(x_train, y_train)

## extracting the best hyperparameters
svm_grid_search.best_params_

{'C': 10, 'gamma': 1, 'kernel': 'rbf'}

In [11]:
## building the model
svm_md = SVC(**svm_grid_search.best_params_, probability = True).fit(x_train, y_train)

## predicting on the testing
svm_preds = svm_md.predict_proba(x_test)[:, 1]

## changing likelihoods to labes
svm_labels = precision_recall_cutoff(y_test, svm_preds)

## printing classification report
print(classification_report(y_test, svm_labels))

              precision    recall  f1-score   support

           0       0.96      0.97      0.97      2286
           1       0.92      0.88      0.90       714

    accuracy                           0.95      3000
   macro avg       0.94      0.93      0.93      3000
weighted avg       0.95      0.95      0.95      3000

